In [1]:
import json;
import urllib2
import os.path
import re;
from pymongo import MongoClient;

def getUserInput():
    FileExists=os.path.isfile('RitoMongo.conf') ;
    res=[];
    if (FileExists):
        with open('RitoMongo.conf') as f:
            for line in f:
                res.append(line.rstrip('\n'));
                print line.rstrip('\n');
    elif (not FileExists):
        SummonerName= raw_input('Enter your Summoner name: ');
        Region  = (raw_input('Enter your region: ')).upper();
        Key = raw_input('Enter your API Key which you retrieved from Riot website: ');
        f = open('RitoMongo.conf','w');
        f.write(SummonerName+'\n'+Region+'\n'+Key);
        f.close();
    return res;

#================================= Main =============================================================================
_InputFields= getUserInput();
SummonerName=_InputFields[0];
Region=_InputFields[1];
Key=_InputFields[2];

BearsAreScary
NA
d8eb3b3a-6401-42af-bb07-a021fd8d9545


In [4]:
res = []
with open('RitoMongo.conf') as f:
    for line in f:
        res.append(line.rstrip('\n'));
Key=res[2];

print(Key)

d8eb3b3a-6401-42af-bb07-a021fd8d9545


In [33]:
from riotwatcher import RiotWatcher, LoLException, error_404, error_429

w = RiotWatcher(key=Key)

# check if we have API calls remaining
print(w.can_make_request())
input_name = 'bearsarescary'
me = w.get_summoner(name=input_name)
print(me)

True
{u'profileIconId': 550, u'summonerLevel': 30, u'revisionDate': 1465084834000, u'id': 20627791, u'name': u'BearsAreScary'}


In [32]:
import pprint
me = w.get_summoner(name='BearsAreScary')
my_ranked_stats = w.get_ranked_stats(me['id'])
pprint.pprint(my_ranked_stats)

{u'champions': [{u'id': 40,
                 u'stats': {u'maxChampionsKilled': 0,
                            u'maxNumDeaths': 9,
                            u'mostChampionKillsPerSession': 0,
                            u'mostSpellsCast': 0,
                            u'totalAssists': 22,
                            u'totalChampionKills': 0,
                            u'totalDamageDealt': 20728,
                            u'totalDamageTaken': 27668,
                            u'totalDeathsPerSession': 9,
                            u'totalDoubleKills': 0,
                            u'totalFirstBlood': 0,
                            u'totalGoldEarned': 11515,
                            u'totalMagicDamageDealt': 12065,
                            u'totalMinionKills': 22,
                            u'totalPentaKills': 0,
                            u'totalPhysicalDamageDealt': 7202,
                            u'totalQuadraKills': 0,
                            u'totalSessionsLost

In [36]:
#Wilson's Lower bound to scale wins by expected win %
#This is code to generate confidence of % of upvotes.
#Shamelessly stolen verbatim from the internet
#https://medium.com/hacking-and-gonzo/how-reddit-ranking-algorithms-work-ef111e33d0d9#.usyxrqagt
# Rewritten code from /r2/r2/lib/db/_sorts.pyx

from math import sqrt

def _confidence(ups, downs):
    n = ups + downs

    if n == 0:
        return 0

    #z = 1.281551565545 #85% confidence interval, this will often suffice
    z = 1.96 #<--- 95% confidence interval
    p = float(ups) / n

    left = p + 1/(2*n)*z*z
    right = z*sqrt(p*(1-p)/n + z*z/(4*n*n))
    under = 1+1/n*z*z

    return (left - right) / under

def confidence(ups, downs):
    if ups + downs == 0:
        return 0
    else:
        return _confidence(ups, downs)
#test:
#confidence(10,0)


In [120]:
my_ranked_stats

{u'champions': [{u'id': 40,
   u'stats': {u'maxChampionsKilled': 0,
    u'maxNumDeaths': 9,
    u'mostChampionKillsPerSession': 0,
    u'mostSpellsCast': 0,
    u'totalAssists': 22,
    u'totalChampionKills': 0,
    u'totalDamageDealt': 20728,
    u'totalDamageTaken': 27668,
    u'totalDeathsPerSession': 9,
    u'totalDoubleKills': 0,
    u'totalFirstBlood': 0,
    u'totalGoldEarned': 11515,
    u'totalMagicDamageDealt': 12065,
    u'totalMinionKills': 22,
    u'totalPentaKills': 0,
    u'totalPhysicalDamageDealt': 7202,
    u'totalQuadraKills': 0,
    u'totalSessionsLost': 0,
    u'totalSessionsPlayed': 1,
    u'totalSessionsWon': 1,
    u'totalTripleKills': 0,
    u'totalTurretsKilled': 0,
    u'totalUnrealKills': 0}},
  {u'id': 41,
   u'stats': {u'maxChampionsKilled': 15,
    u'maxNumDeaths': 10,
    u'mostChampionKillsPerSession': 15,
    u'mostSpellsCast': 0,
    u'totalAssists': 162,
    u'totalChampionKills': 116,
    u'totalDamageDealt': 4107034,
    u'totalDamageTaken': 511527

In [149]:
from collections import defaultdict

my_ranked_dict = defaultdict(list)
player_ranked_dict = defaultdict(float)

for item in my_ranked_stats['champions']:
    #if item['id'] != 0:
    won_games = float(item['stats']['totalSessionsWon'])
    lost_games = float(item['stats']['totalSessionsLost'])
    played_games = float(item['stats']['totalSessionsPlayed'])
    win_perc = round(won_games / played_games,2)
    my_ranked_dict[item['id']].append(int(played_games))
    player_ranked_dict[item['id']] = int(played_games)
    my_ranked_dict[item['id']].append(int(win_perc*100))
del(my_ranked_dict[0])
    #my_ranked_dict[item['id']].append(round(confidence(won_games,lost_games),2))
#    pprint.pprint([item['id'],played_games,win_perc,confidence(won_games,lost_games)])
my_ranked_dict

top_five_id = sorted(my_ranked_dict, key=my_ranked_dict.get, reverse=True)[:5]
top_five_values = []
for index, item in enumerate(top_five_id):
     top_five_values.append([item,my_ranked_dict[item]])
        
top_five_values[0][1][0]

19

In [39]:
match_list = w.get_match_history(summoner_id=me['id'])

In [40]:
#match_list[match_list['season'] == 'SEASON2016']:
#    pprint.pprint(match)
from collections import defaultdict, Counter
curr_ranked_play = defaultdict(list)

for match in match_list['matches']:
    if match['season'] == 'SEASON2016':
        print match
        #curr_ranked_play[match['champion']].append(match['lane'])

{u'lane': u'TOP', u'champion': 41, u'platformId': u'NA1', u'season': u'SEASON2016', u'region': u'NA', u'matchId': 2198981955, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'SOLO', u'timestamp': 1464312997962}
{u'lane': u'MID', u'champion': 41, u'platformId': u'NA1', u'season': u'SEASON2016', u'region': u'NA', u'matchId': 2198917800, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'SOLO', u'timestamp': 1464311185555}
{u'lane': u'BOTTOM', u'champion': 143, u'platformId': u'NA1', u'season': u'SEASON2016', u'region': u'NA', u'matchId': 2198902481, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_SUPPORT', u'timestamp': 1464308770157}
{u'lane': u'BOTTOM', u'champion': 143, u'platformId': u'NA1', u'season': u'SEASON2016', u'region': u'NA', u'matchId': 2198846636, u'queue': u'TEAM_BUILDER_DRAFT_RANKED_5x5', u'role': u'DUO_SUPPORT', u'timestamp': 1464305991252}
{u'lane': u'BOTTOM', u'champion': 143, u'platformId': u'NA1', u'season': u'SEASON2016', u'region': u'NA', u'ma

In [54]:
tags = w.static_get_champion_list(champ_data='tags')
tags

{u'data': {u'Aatrox': {u'id': 266,
   u'key': u'Aatrox',
   u'name': u'Aatrox',
   u'tags': [u'Fighter', u'Tank'],
   u'title': u'the Darkin Blade'},
  u'Ahri': {u'id': 103,
   u'key': u'Ahri',
   u'name': u'Ahri',
   u'tags': [u'Mage', u'Assassin'],
   u'title': u'the Nine-Tailed Fox'},
  u'Akali': {u'id': 84,
   u'key': u'Akali',
   u'name': u'Akali',
   u'tags': [u'Assassin'],
   u'title': u'the Fist of Shadow'},
  u'Alistar': {u'id': 12,
   u'key': u'Alistar',
   u'name': u'Alistar',
   u'tags': [u'Tank', u'Support'],
   u'title': u'the Minotaur'},
  u'Amumu': {u'id': 32,
   u'key': u'Amumu',
   u'name': u'Amumu',
   u'tags': [u'Tank', u'Mage'],
   u'title': u'the Sad Mummy'},
  u'Anivia': {u'id': 34,
   u'key': u'Anivia',
   u'name': u'Anivia',
   u'tags': [u'Mage', u'Support'],
   u'title': u'the Cryophoenix'},
  u'Annie': {u'id': 1,
   u'key': u'Annie',
   u'name': u'Annie',
   u'tags': [u'Mage'],
   u'title': u'the Dark Child'},
  u'Ashe': {u'id': 22,
   u'key': u'Ashe',
   u'n

In [158]:
tag_dict = defaultdict(list)
champ_dict = defaultdict(list)
tag_list = ['Fighter','Mage','Tank','Support','Marksman','Assassin']
tag_dict[0].append(u'All')
tag_dict[0].append(tag_list)
for key, value in tags['data'].iteritems():
    tag_dict[value['id']].append(value['name'])
    champ_dict[value['id']] = value['name']
    tag_dict[value['id']].append(value['tags'])

pickle.dump(champ_dict,open('Riot-API-Code/role_id/champion_id_to_name.pkl','w'))    
champs = pickle.load(open('Riot-API-Code/role_id/champion_id_to_name.pkl'))

In [159]:
from itertools import chain
import pandas as pd

columns = ['Name', 'Tags', 'Games Played', 'Win Percentage', 'Wilson Win Percentage']
dicts = [champs, player_ranked_dict]
super_dict = {}

for d in dicts:
    for k, v in d.iteritems():
        super_dict.setdefault(k, []).extend(v)
        
# player_df = pd.DataFrame.from_dict(data=super_dict,orient='index')
# player_df.columns = columns
# player_df.fillna(0,inplace=True)
# player_df

player_ranked_dict

TypeError: 'int' object is not iterable

In [4]:
import glob
import numpy as np
try:
    import cPickle as pickle
except:
    import pickle

game_data = [pickle.load(open(x,'rb')) for x in glob.glob('*.pkl')]
game_array = np.asarray(game_data)

game_array.shape

(54,)

In [123]:
import pandas as pd
import pprint

true_game_list = []
[true_game_list.extend(x) for x in game_data]
true_game_list

[{'losingChamps': [48, 131, 81, 45, 40],
  'mapId': 11,
  'matchCreation': 1464312997962,
  'matchId': 2198981955,
  'matchMode': u'CLASSIC',
  'matchVersion': u'6.10.144.9736',
  'queueType': u'TEAM_BUILDER_DRAFT_RANKED_5x5',
  'winningChamps': [16, 104, 99, 236, 41]},
 {'losingChamps': [143, 67, 121, 136, 24],
  'mapId': 11,
  'matchCreation': 1464236474686,
  'matchId': 2198436980,
  'matchMode': u'CLASSIC',
  'matchVersion': u'6.10.144.9736',
  'queueType': u'TEAM_BUILDER_DRAFT_RANKED_5x5',
  'winningChamps': [154, 54, 51, 245, 89]},
 {'losingChamps': [34, 76, 104, 98, 143],
  'mapId': 11,
  'matchCreation': 1464233685551,
  'matchId': 2198440621,
  'matchMode': u'CLASSIC',
  'matchVersion': u'6.10.144.9736',
  'queueType': u'TEAM_BUILDER_DRAFT_RANKED_5x5',
  'winningChamps': [4, 40, 113, 15, 39]},
 {'losingChamps': [106, 63, 222, 7, 13],
  'mapId': 11,
  'matchCreation': 1464227940846,
  'matchId': 2198317220,
  'matchMode': u'CLASSIC',
  'matchVersion': u'6.10.144.9736',
  'queue

In [164]:
basic_info = w.get_summoner(name='bearsarescary')
basic_info

{u'id': 20627791,
 u'name': u'BearsAreScary',
 u'profileIconId': 550,
 u'revisionDate': 1465084834000,
 u'summonerLevel': 30}

In [162]:
#GET INPUT FROM TEXT BOX
def get_summoner_history(summoner_name):

    basic_info = w.get_summoner(name=summoner_name)
    match_history = w.get_ranked_stats(basic_info['id'])
    player_ranked_dict = defaultdict(list)

    for item in match_history['champions']:
        played_games = float(item['stats']['totalSessionsPlayed'])
        won_games = float(item['stats']['totalSessionsWon'])
        win_perc = round(won_games / played_games,2)
        #player_ranked_dict[item['id']] = int(played_games)
        player_ranked_dict[item['id']].append(int(played_games))
        player_ranked_dict[item['id']].append(int(win_perc*100))
        #player_ranked_dict[item['id']].append(win_perc)
    del(player_ranked_dict[0])

    top_five_id = sorted(player_ranked_dict, key=player_ranked_dict.get, reverse=True)[:5]
    top_five_values = []
    for index, item in enumerate(top_five_id):
         top_five_values.append([item,player_ranked_dict[item]])

    return top_five_values

In [185]:
def get_ranked_tier(summoner_name):
    basic_info = w.get_summoner(name=summoner_name)
    player_id = str(basic_info['id'])
    league_entry = w.get_league_entry(summoner_ids=[player_id])
    tier = league_entry[str(player_id)][0]['tier'].lower()
    division = league_entry[player_id][0]['entries'][0]['division']
    division = division.replace("I","1").replace("II","2").replace("I","3")\
    .replace("IV","4").replace("V","5")
    tier_division = [tier, division]
    return tier_division

In [186]:
get_ranked_tier('bearsarescary')

[u'gold', u'1']

[{u'entries': [{u'division': u'I',
    u'isFreshBlood': False,
    u'isHotStreak': False,
    u'isInactive': False,
    u'isVeteran': False,
    u'leaguePoints': 75,
    u'losses': 27,
    u'playerOrTeamId': u'20627791',
    u'playerOrTeamName': u'BearsAreScary',
    u'playstyle': u'NONE',
    u'wins': 35}],
  u'name': u"Xerath's Renegades",
  u'queue': u'RANKED_SOLO_5x5',
  u'tier': u'GOLD'}]